<a href="https://colab.research.google.com/github/dinky-coder/dinky-coder/blob/main/Advanced_Agent_Interaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Selective Memory Sharing: Using LLM Understanding for Context Selection

In [2]:
# Placeholder definitions for missing components
# Replace these with actual imports or definitions from your framework

def register_tool(description):
    def decorator(func):
        # In a real scenario, this would register the function as a tool
        print(f"Registering tool: {func.__name__} with description: {description}")
        return func
    return decorator

class ActionContext:
    def get_agent_registry(self):
        print("Placeholder: Getting agent registry")
        # Return a mock or actual agent registry
        return MockAgentRegistry()

    def get_memory(self):
        print("Placeholder: Getting memory")
        # Return a mock or actual memory object
        return Memory()

def prompt_llm_for_json(action_context, schema, prompt):
    print("Placeholder: Prompting LLM for JSON")
    # In a real scenario, this would interact with an LLM
    # Returning a dummy response based on the selection_schema
    return {
        "selected_memories": ["mem_0", "mem_2"], # Example selection
        "reasoning": "These memories are relevant to the task."
    }

class Memory:
    def __init__(self):
        self.items = [] # Using a list to store memory items for this example

    def add_memory(self, item):
        print(f"Placeholder: Adding memory item: {item}")
        self.items.append(item)

    # Adding a simple getter for items for demonstration
    def get_items(self):
      return self.items

# Mock Agent Registry and Agent for demonstration purposes
class MockAgentRegistry:
    def get_agent(self, agent_name):
        print(f"Placeholder: Getting agent: {agent_name}")
        return MockAgent(agent_name)

class MockAgent:
    def __init__(self, name):
        self.name = name

    def __call__(self, user_input, memory):
        print(f"Placeholder: Running agent {self.name} with input: {user_input} and memory: {memory.get_items()}")
        # Simulate agent processing and returning a result memory
        result_memory = Memory()
        result_memory.add_memory({"type": "result", "content": f"Result from {self.name} for task: {user_input}"})
        return result_memory


@register_tool(description="Delegate a task to another agent with selected context")
def call_agent_with_selected_context(action_context: ActionContext,
                                   agent_name: str,
                                   task: str) -> dict:
    """Call agent with LLM-selected relevant memories."""
    agent_registry = action_context.get_agent_registry()
    agent_run = agent_registry.get_agent(agent_name)

    # Get current memory and add IDs
    current_memory = action_context.get_memory()
    memory_with_ids = []
    # Adding some dummy memory items for demonstration
    dummy_memory_items = [
        {"type": "user", "content": "Initial user input"},
        {"type": "system", "content": "Some system information"},
        {"type": "user", "content": "Another user query"}
    ]
    current_memory.items.extend(dummy_memory_items) # Add dummy items to the current memory

    for idx, item in enumerate(current_memory.items):
        memory_with_ids.append({
            **item,
            "memory_id": f"mem_{idx}"
        })

    # Create schema for memory selection
    selection_schema = {
        "type": "object",
        "properties": {
            "selected_memories": {
                "type": "array",
                "items": {
                    "type": "string",
                    "description": "ID of a memory to include"
                }
            },
            "reasoning": {
                "type": "string",
                    "description": "Explanation of why these memories were selected"
            }
        },
        "required": ["selected_memories", "reasoning"]
    }

    # Format memories for LLM review
    memory_text = "\n".join([
        f"Memory {m['memory_id']}: {m['content']}"
        for m in memory_with_ids
    ])

    # Ask LLM to select relevant memories
    selection_prompt = f"""Review these memories and select the ones relevant for this task:

Task: {task}

Available Memories:
{memory_text}

Select memories that provide important context or information for this specific task.
Explain your selection process."""

    # Self-prompting magic to find the most relevant memories
    selection = prompt_llm_for_json(
        action_context=action_context,
        schema=selection_schema,
        prompt=selection_prompt
    )

    # Create filtered memory from selection
    filtered_memory = Memory()
    selected_ids = set(selection["selected_memories"])
    for item in memory_with_ids:
        if item["memory_id"] in selected_ids:
            # Remove the temporary memory_id before adding
            item_copy = item.copy()
            del item_copy["memory_id"]
            filtered_memory.add_memory(item_copy)

    # Run the agent with selected memories
    result_memory = agent_run(
        user_input=task,
        memory=filtered_memory
    )

    # Add results and selection reasoning to original memory
    current_memory.add_memory({
        "type": "system",
        "content": f"Memory selection reasoning: {selection['reasoning']}"
    })

    for memory_item in result_memory.items:
        current_memory.add_memory(memory_item)

    return {
        "result": result_memory.items[-1].get("content", "No result"),
        "shared_memories": len(filtered_memory.items),
        "selection_reasoning": selection["reasoning"]
    }

Registering tool: call_agent_with_selected_context with description: Delegate a task to another agent with selected context
